In [2]:

# Common imports


from sklearn.model_selection import train_test_split
from PIL import Image
import pandas as pd
import numpy as np
import os

# to make this notebook's output stable across runs
def reset_graph(seed=42):
    tf.reset_default_graph()
    tf.set_random_seed(seed)
    np.random.seed(seed)

# To plot pretty figures
%matplotlib inline
import matplotlib
import matplotlib.pyplot as plt
plt.rcParams['axes.labelsize'] = 14
plt.rcParams['xtick.labelsize'] = 12
plt.rcParams['ytick.labelsize'] = 12

# Where to save the figures
PROJECT_ROOT_DIR = "."
CHAPTER_ID = "cnn"

def save_fig(fig_id, tight_layout=True):
    path = os.path.join(PROJECT_ROOT_DIR, "images", CHAPTER_ID, fig_id + ".png")
    print("Saving figure", fig_id)
    if tight_layout:
        plt.tight_layout()
    plt.savefig(path, format='png', dpi=300)
os.chdir("/Users/giang/Documents/MIDS/w207 projects/final project")

In [3]:
train_labels = pd.read_csv("Kaggle data/train_labels.csv")['invasive'].values
len(train_labels)


2295

In [4]:
from numpy import array

train_data= np.zeros(shape=(len(train_labels), 866, 1154, 3), dtype=np.uint8)
for i in range(train_data.shape[0]):
    im = Image.open("Kaggle data/train/"+str(i+1)+".jpg")
    # open image file and store in variable `im`, then
    train_data[i] = np.array(im)
train_data.shape

(2295, 866, 1154, 3)

In [5]:
X_train = train_data[:1600]
X_dev = train_data[1600:]
y_train = train_labels[:1600]
y_dev = train_labels[1600:]




In [6]:
import random as rd
def next_batch(batch_size):
    max_ind = len(y_train)
    start = rd.randint(0, max_ind-batch_size)
    end = start+batch_size 
    return train_data[start:end],y_train[start:end] 
 
    


In [7]:
import tensorflow as tf


In [ ]:
height = 866
width = 1154
channels = 3
n_inputs = height * width
batch_size=100

conv1_fmaps = 32
conv1_ksize = 3
conv1_stride = 1
conv1_pad = "SAME"

conv2_fmaps = 64
conv2_ksize = 3
conv2_stride = 2
conv2_pad = "SAME"

pool3_fmaps = conv2_fmaps

n_fc1 = 64
n_outputs = 10

reset_graph()

with tf.name_scope("inputs"):
    X = tf.placeholder(tf.float32, shape=[None, height, width, channels], name="X")
    X_resized = tf.image.resize_images(X, [280, 280])
    print('X_resize shape now',X_resized.shape)

    y = tf.placeholder(tf.int32, shape=[None], name="y")

conv1 = tf.layers.conv2d(X_resized, filters=conv1_fmaps, kernel_size=conv1_ksize,
                         strides=conv1_stride, padding=conv1_pad,
                         activation=tf.nn.relu, name="conv1")
conv2 = tf.layers.conv2d(conv1, filters=conv2_fmaps, kernel_size=conv2_ksize,
                         strides=conv2_stride, padding=conv2_pad,
                         activation=tf.nn.relu, name="conv2")

with tf.name_scope("pool3"):
    pool3 = tf.nn.max_pool(conv2, ksize=[1, 2, 2, 1], strides=[1, 2, 2, 1], padding="VALID")
    pool3_flat = tf.reshape(pool3, shape=[-1, pool3_fmaps * 70 * 70])
    print('Pool3 shape is: ', pool3.shape)
    
    print('Pool3 float shape is: ', pool3_flat.shape)
    

with tf.name_scope("fc1"):
    fc1 = tf.layers.dense(pool3_flat, n_fc1, activation=tf.nn.relu, name="fc1")

with tf.name_scope("output"):
    logits = tf.layers.dense(fc1, n_outputs, name="output")
    Y_proba = tf.nn.softmax(logits, name="Y_proba")

with tf.name_scope("train"):
    xentropy = tf.nn.sparse_softmax_cross_entropy_with_logits(logits=logits, labels=y)
    loss = tf.reduce_mean(xentropy)
    optimizer = tf.train.AdamOptimizer()
    training_op = optimizer.minimize(loss)

with tf.name_scope("eval"):
    correct = tf.nn.in_top_k(logits, y, 1)
    accuracy = tf.reduce_mean(tf.cast(correct, tf.float32))

with tf.name_scope("init_and_save"):
    init = tf.global_variables_initializer()
    saver = tf.train.Saver()



X_resize shape now (?, 280, 280, 3)
Pool3 shape is:  (?, 70, 70, 64)
Pool3 float shape is:  (?, 313600)


In [ ]:
n_epochs = 10
batch_size = 100

with tf.Session() as sess:
    init.run()
    for epoch in range(n_epochs):
        for iteration in range(20):
            X_batch, y_batch = next_batch(batch_size)
            sess.run(training_op, feed_dict={X: X_batch, y: y_batch})
        acc_train = accuracy.eval(feed_dict={X: X_batch, y: y_batch})
        acc_test = accuracy.eval(feed_dict={X: X_dev, y:y_dev})
        print(epoch, "Train accuracy:", acc_train, "Test accuracy:", acc_test)

        save_path = saver.save(sess, "./my_isd_model")